In [1]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.4ubuntu1).
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is already the newest version (1:2.17.1-1ubuntu0.12).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
tar is already the newest version (1.29b-2ubuntu0.3).
unzip is already the newest version (6.0-21ubuntu1.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.1).
zlib1g-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  autoconf automake autopoint autotools-dev debhelper dh-autoreconf
  dh-strip-nondeterminism file freepats gettext gettext-base gir1.2

In [2]:
!pip install vizdoom
!pip install ray 
!pip install ray['rllib']
!pip install Ipython --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for vizdoom: filename=vizdoom-1.1.13-cp37-cp37m-linux_x86_64.whl size=14101153 sha256=9e53b8c7ebb9c3b44e3043b514f8bc319368e4b22226ed5b7484b128fe43b622
  Stored in directory: /root/.cache/pip/wheels/ac/37/ae/8e648023f66bb4c473701f94ce126032ff39ad9759ca0645a7
Successfully built vizdoom
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 68 kB/s 
     |████████████████████████████████| 8.8 MB 49.6 MB/s 
     |████████████████████████████████| 4.1 MB 49.3 MB/s 
     |████████████████████████████████| 468 kB 70.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Unin

In [3]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project'
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(system_path)

Mounted at /content/drive


In [4]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/GitHub/INM363-Project']


In [5]:
from src.vizdoom_gym.envs.VizDoomEnv import VizdoomEnv
from src.vizdoom_gym.envs.VizDoomEnv_def import VizDoomVeryDenseReward

In [6]:
from ray.tune.registry import register_env
import gym
import os
import ray
import ray.rllib.agents.ppo as ppo
import shutil

In [7]:
print("Hello world")

#chkpt_root = "tmp/exa"
chkpt_root = "'/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_sparse_ppo"
shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)

print("here 1")

ray_results = "{}/ray_results/".format(os.getenv("HOME"))
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

print("here 2")

ray.shutdown()
print("Shutdown ray")
# start Ray -- add `local_mode=True` here for debugging
ray.init(ignore_reinit_error=True, local_mode=True)
print("here 3: initialized ray")

# register the custom environment
select_env = "VizDoomVeryDenseReward-v0"

register_env(select_env, lambda config: VizDoomVeryDenseReward())
#register_env(select_env, lambda config: VizdoomEnv())

print("registered environment")


Hello world
here 1
here 2
Shutdown ray


/usr/local/lib/python3.7/dist-packages/ray/_private/client_mode_hook.py:105: DeprecationWarning: DeprecationWarning: local mode is an experimental feature that is no longer maintained and will be removed in the future.For debugging consider using Ray debugger. 
  return func(*args, **kwargs)
2022-08-30 14:23:00,542	INFO worker.py:1518 -- Started a local Ray instance.


here 3: initialized ray
registered environment


In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [9]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [10]:
# configure the environment and create agent
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config["num_workers"] = 1
config["framework"] = "torch"
#config["horizon"] = 50
agent = ppo.PPOTrainer(config, env=select_env)

print("created agent")

2022-08-30 14:23:02,365	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-08-30 14:23:02,372	INFO algorithm.py:354 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/usr/local/lib/python3.7/dist-packages/ray/_private/ray_option_utils.py:273: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  stacklevel=1,
2022-08-30 14:23:02,493	WARNING worker.py:831 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:actor_name:

:actor_name:RolloutWorker
config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/easy_sparse_reward.wad


2022-08-30 14:23:03,672	WARNING env.py:143 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-08-30 14:23:04,845	WARNING util.py:65 -- Install gputil for GPU system monitoring.


created agent


In [11]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 10

print("started training loop")

# train a policy with RLlib using PPO
for n in range(n_iter):
  
    result = agent.train()

    #print(i)
    #print(result)
    chkpt_file = agent.save(chkpt_root)

    print(status.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"],
        chkpt_file
    ))

print("ending training loop")

ray.shutdown()
print("shutdown ray")

started training loop
 1 reward  -0.10/ 47.07/ 99.95 len 19.02 saved '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_sparse_ppo/checkpoint_000001
 2 reward  -0.10/ 82.30/ 99.95 len 14.70 saved '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_sparse_ppo/checkpoint_000002
 3 reward  -0.10/ 94.11/ 99.95 len 12.99 saved '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_sparse_ppo/checkpoint_000003


KeyboardInterrupt: ignored

In [12]:
print("hello")

hello


In [13]:
# examine the trained policy
policy = agent.get_policy()
model = policy.model

In [14]:
model

VisionNetwork(
  (_logits): SlimConv2d(
    (_model): Sequential(
      (0): ZeroPad2d((0, 0, 0, 0))
      (1): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (_convs): Sequential(
    (0): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((5, 6, 5, 5))
        (1): Conv2d(3, 16, kernel_size=(12, 16), stride=(7, 9))
        (2): ReLU()
      )
    )
    (1): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((1, 1, 1, 2))
        (1): Conv2d(16, 32, kernel_size=(6, 6), stride=(4, 4))
        (2): ReLU()
      )
    )
    (2): SlimConv2d(
      (_model): Sequential(
        (0): Conv2d(32, 256, kernel_size=(9, 9), stride=(1, 1))
        (1): ReLU()
      )
    )
  )
  (_value_branch_separate): Sequential(
    (0): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((5, 6, 5, 5))
        (1): Conv2d(3, 16, kernel_size=(12, 16), stride=(7, 9))
        (2): ReLU()
      )
    )
    (1): SlimConv2d(
      (_model): Sequential(
        (0): 

In [15]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project/src' #need this to load vizdoom module 
sys.path.append(system_path)

In [16]:
agent.restore(chkpt_file)
env = gym.make('VizDoomVeryDenseReward-v0')

2022-08-30 15:25:39,838	INFO trainable.py:669 -- Restored on 172.28.0.2 from checkpoint: '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_sparse_ppo/checkpoint_000003
2022-08-30 15:25:39,840	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 3, '_timesteps_total': None, '_time_total': 3251.403177022934, '_episodes_total': 790}
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment VizdoomMyWayHome-v0
  logger.warn(f"Overriding environment {id}")
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:595: UserWarning: WARN: Overriding environment VizDoomVeryDenseReward-v0
  logger.warn(f"Overriding environment {id}")


config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/easy_sparse_reward.wad


In [20]:
state = env.reset()
sum_reward = 0
n_step = 20

for step in range(n_step):
    action = agent.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward

    #env.render()

    if done == 1:
        # report at the end of each episode
        print("cumulative reward", sum_reward)
        print(f"total steps: {step}")
        state = env.reset()
        sum_reward = 0
        break

if done == 0:
  print("cumulative reward", sum_reward)
  print(f"total steps: {step}")


cumulative reward 99.95499999999998
total steps: 11


In [ ]:
#chkpt_root = "'/content/drive/MyDrive/GitHub/INM363-Project/tmp/exa"
#shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)

In [ ]:
"""
print(result)

chkpt_file = agent.save(chkpt_root)

print(status.format(
  n + 1,
  result["episode_reward_min"],
  result["episode_reward_mean"],
  result["episode_reward_max"],
  result["episode_len_mean"],
  chkpt_file
))
"""